In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


#1. Mount to OneDrive


#2. Mount to Google Drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
callhome_dir = "/content/gdrive/MyDrive/callhome"
callhome1_dir = "/content/gdrive/MyDrive/Speaker_Diarization/callhome1"
callhome2_dir = "/content/gdrive/MyDrive/Speaker_Diarization/callhome2"
pretrained_model = "/content/gdrive/MyDrive/Speaker_Diarization/ENND model/eend_swb-sre_alimeeting.nnet.npz"




#3. Create environment

In [4]:
!rm -r /content/EEND
!git clone https://github.com/lekynam2000/EEND.git
!git clone https://github.com/lekynam2000/Speaker_Diarization.git

rm: cannot remove '/content/EEND': No such file or directory
Cloning into 'EEND'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 331 (delta 108), reused 183 (delta 74), pack-reused 99
Receiving objects: 100% (331/331), 51.79 MiB | 13.16 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Cloning into 'Speaker_Diarization'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15 (delta 2), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [5]:
!pip install kora -q
import kora.install.kaldi

     |████████████████████████████████| 57 kB 2.1 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 


In [6]:
sd_dir = "/content/EEND"
callhome_rttm = "/content/Speaker_Diarization/metadata/callhome/callhome.rttm"

In [7]:
%cd $sd_dir

/content/EEND


In [8]:
!cd tools && make KALDI=/opt/kaldi
!sudo apt-get install -y flac
!sudo apt install sox

ln -s /opt/kaldi kaldi
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda3.sh
--2022-03-11 09:59:36--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-03-11 09:59:36--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[==================

In [9]:
%cd egs/callhome/v1

/content/EEND/egs/callhome/v1


In [10]:
!/content/EEND/tools/miniconda3/envs/eend/bin/pip install ipykernel pathlib
from pathlib import Path

     |████████████████████████████████| 128 kB 4.0 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 102 kB 9.3 MB/s 
     |████████████████████████████████| 428 kB 58.0 MB/s 
     |████████████████████████████████| 793 kB 44.1 MB/s 
     |████████████████████████████████| 130 kB 63.7 MB/s 
     |████████████████████████████████| 1.9 MB 49.3 MB/s 
     |████████████████████████████████| 380 kB 47.7 MB/s 
     |████████████████████████████████| 59 kB 5.4 MB/s 
     |████████████████████████████████| 1.6 MB 40.6 MB/s 
     |████████████████████████████████| 1.1 MB 51.2 MB/s 
     |████████████████████████████████| 100 kB 8.1 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14363 sha256=1bac71f7cfddde488c2f8110666cb9904a324ceeed247699fe5b360012feb08f
  Stored in directory: /root/.cache/pip/wheel

In [11]:
local_callhome = "data/eval/callhome"
local_model = "model/eend_swb-sre_alimeeting.nnet.npz"

In [12]:
!mkdir -p "data/eval"
!mkdir "model"
!cp -R "$callhome1_dir" "$local_callhome"
!cp "$pretrained_model" "$local_model"

In [13]:
!mkdir -p "/home/lcy/corpus/LDC2001S97"
!cp -a "$callhome_dir/sph/." "/home/lcy/corpus/LDC2001S97" 

In [14]:
import soundfile as sf
import io
import subprocess
import librosa


In [15]:
print(sf.available_formats())

{'AIFF': 'AIFF (Apple/SGI)', 'AU': 'AU (Sun/NeXT)', 'AVR': 'AVR (Audio Visual Research)', 'CAF': 'CAF (Apple Core Audio File)', 'FLAC': 'FLAC (Free Lossless Audio Codec)', 'HTK': 'HTK (HMM Tool Kit)', 'SVX': 'IFF (Amiga IFF/SVX8/SV16)', 'MAT4': 'MAT4 (GNU Octave 2.0 / Matlab 4.2)', 'MAT5': 'MAT5 (GNU Octave 2.1 / Matlab 5.0)', 'MPC2K': 'MPC (Akai MPC 2k)', 'OGG': 'OGG (OGG Container format)', 'PAF': 'PAF (Ensoniq PARIS)', 'PVF': 'PVF (Portable Voice Format)', 'RAW': 'RAW (header-less)', 'RF64': 'RF64 (RIFF 64)', 'SD2': 'SD2 (Sound Designer II)', 'SDS': 'SDS (Midi Sample Dump Standard)', 'IRCAM': 'SF (Berkeley/IRCAM/CARL)', 'VOC': 'VOC (Creative Labs)', 'W64': 'W64 (SoundFoundry WAVE 64)', 'WAV': 'WAV (Microsoft)', 'NIST': 'WAV (NIST Sphere)', 'WAVEX': 'WAVEX (Microsoft)', 'WVE': 'WVE (Psion Series 3)', 'XI': 'XI (FastTracker 2)'}


In [16]:
rec2dur = {}
lines = [line.strip().split(" ") for line in open(f"{callhome2_dir}/wav.scp")]
for line in lines:
  f = sf.SoundFile(f"{callhome_dir}/wav/{line[0]}.wav") 
  dur = f.frames/f.samplerate
  rec2dur[line[0]] = dur
with open(f"{callhome2_dir}/reco2dur","w+") as file:
  for recid in rec2dur.keys():
    file.write(f"{recid} {rec2dur[recid]}\n")

In [17]:
rec2dur = {}
lines = [line.strip().split(" ") for line in open(f"{callhome1_dir}/wav.scp")]
for line in lines:
  f = sf.SoundFile(f"{callhome_dir}/wav/{line[0]}.wav") 
  dur = f.frames/f.samplerate
  rec2dur[line[0]] = dur
with open(f"{callhome1_dir}/reco2dur","w+") as file:
  for recid in rec2dur.keys():
    file.write(f"{recid} {rec2dur[recid]}\n")

In [18]:
lines_callhome1 = [line.strip().split(" ") for line in open(f"{callhome1_dir}/wav.scp")]
lines_callhome2 = [line.strip().split(" ") for line in open(f"{callhome2_dir}/wav.scp")]
print(f"ch1: {len(lines_callhome1)}")
print(f"ch2: {len(lines_callhome2)}")

ch1: 250
ch2: 250
